#NanoFore


## STR detection

The STR profile is generated using two approaches; one based on the length of the STR regions the other on the  actual sequence.

- Part 1: Splicing the reads into subreads 
- Part 2A: Length based profiling
- Part 2B: Sequence based profiling 

For all below replace '/media/sf_vm_shared/nanopore' with the root path to the project directory.
In my case, the fast5 files are in a subdirectory 'pass', results will be written into a subdirectory 'results' and any other files needed are located in the root of the project.


## PART 1: Splicing read into subreads. 

- Loading Primers
- Loading Sequencing data
- Using fuzzy Regex to identify the primers and slice the reads.

###Checking current fast5 format

In [1]:
import h5py

# Example f5 file
f5 = h5py.File('/media/genomics/raw_data/FW01C250_20160531_FN_MN15671_sequencing_run_SNPforID_310516_26276_ch101_read1000_strand.fast5')

# Walk the file structure
def printname(name):
  print(name,'(group)' if type(f5[name]) == h5py._hl.group.Group else '')

f5.visit(printname)

Analyses (group)
Analyses/Basecall_1D_000 (group)
Analyses/Basecall_1D_000/BaseCalled_complement (group)
Analyses/Basecall_1D_000/BaseCalled_complement/Events 
Analyses/Basecall_1D_000/BaseCalled_complement/Fastq 
Analyses/Basecall_1D_000/BaseCalled_complement/Model 
Analyses/Basecall_1D_000/BaseCalled_template (group)
Analyses/Basecall_1D_000/BaseCalled_template/Events 
Analyses/Basecall_1D_000/BaseCalled_template/Fastq 
Analyses/Basecall_1D_000/BaseCalled_template/Model 
Analyses/Basecall_1D_000/Configuration (group)
Analyses/Basecall_1D_000/Configuration/aggregator (group)
Analyses/Basecall_1D_000/Configuration/basecall_1d (group)
Analyses/Basecall_1D_000/Configuration/basecall_2d (group)
Analyses/Basecall_1D_000/Configuration/calibration_strand (group)
Analyses/Basecall_1D_000/Configuration/components (group)
Analyses/Basecall_1D_000/Configuration/general (group)
Analyses/Basecall_1D_000/Configuration/hairpin_align (group)
Analyses/Basecall_1D_000/Configuration/post_processing (gro

###Extracting Fastq file from Fast5 files

In [3]:
import h5py, glob

infiles          = glob.glob("/media/genomics/raw_data/*fast5")
outfile          = "/media/genomics/raw_data/Nanopore_data/ligatedSTR.fastq"
outfile          = open(outfile, 'wt')
withoutFQcounter = 0

for f5file in infiles:
  f5 = h5py.File(f5file)
  try:
    fq = f5["Analyses/Basecall_2D_000/BaseCalled_2D/Fastq"]
    outfile.write(fq.value.decode())
  except KeyError:
    withoutFQcounter += 1 
  f5.close()
           
print(withoutFQcounter, "fast5 files without called fastq sequence")
outfile.close()

0 fast5 files without called fastq sequence


###Counting primers

In [ ]:
%matplotlib inline

from itertools import count
import matplotlib.pyplot as plt
import sys, os
sys.path.append('/media/genomics/raw_data/')
from MyFLq import complement

fastq = open("/media/genomics/raw_data/Nanopore_data/ligatedSTR.fastq")
c     = count(0)
reads = [line for line in fastq if next(c)%4 == 1]

# Plot length histogram
plt.hist([len(r) for r in reads], bins=1000)
plt.xlabel("Length (Bp)",size=33)
plt.ylabel("Frequency (n)",size=33)
plt.savefig('myfig.svg')


allDataOneString = ' '.join(reads)

# Primers
primers = {}
for line in open("/media/genomics/raw_data/primers2.csv"):
  if line.startswith('#'): continue
  line             = line.strip().split(',')
  primers[line[0]] = {'for':  [line[1],allDataOneString.count(line[1])],
                      'forc': [complement(line[1]),allDataOneString.count(complement(line[1]))],
                      'rev':  [line[2],allDataOneString.count(line[2])],
                      'revc': [complement(line[2]),allDataOneString.count(complement(line[2]))],
                     }

for p in primers:
  print(p,
        primers[p]['for'][1],
        primers[p]['forc'][1],
        primers[p]['rev'][1],
        primers[p]['revc'][1])


## Part2A: Analyzing full reads and creating a length based profile

In [ ]:
# Analyzing full reads
%matplotlib inline

from itertools import count
from math import floor,ceil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os.path import expanduser as eu
import sys, os
sys.path.append(eu("~/repos/myflq/src/"))
from MyFLq import complement, calculateAlleleNumber, Locus

loci = Locus.makeLocusDict(('csv',eu('~/repos/Nanofore/lociConfiguration.csv')))

exit()
#Functions
def bpTOfloat(bpLength,locusType):
    return int(bpLength)+(int(str(bpLength).split('.')[1]) if '.' in str(bpLength) else 0)/locusType

def floatTObp(floatLength,locusType):
    return int(floatLength)+(int(str(floatLength).split('.')[1]) if '.' in str(floatLength) else 0)*locusType

def calculateOriginalLength(repeats,locusType,refsize,refrepeats):
    fullRefRepeats,partialRefRepeat = str(float(refrepeats)).split('.')
    fullRepeats,partialRepeat = str(float(repeats)).split('.')
    return refsize+locusType*(int(fullRepeats)-int(fullRefRepeats))+(int(partialRepeat)-int(partialRefRepeat))

def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

def gaus2(x,a_1,x0_1,sigma_1,a_2,x0_2,sigma_2):
    return a_1*np.exp(-(x-x0_1)**2/(2*sigma_1**2))+a_2*np.exp(-(x-x0_2)**2/(2*sigma_2**2))

# Main class
class LigatedRead:
    """
    Class that represents a ligated read of subreads.
    Methods allow to extract the subreads for further processing.
    The initial read can contain '&', which are divisions between
    already known subreads.
    """
    def __init__(self, read, loci,minSize1read=70, maxSize1read=500, maxPrimerErrors=0):
        self.read = read
        for l in loci:
            # Prep loci for use with MyFLq.calculateAlleleNumber
            loci[l]['ref_length']       = len(loci[l]['ref_sequence'])
            loci[l]['ref_alleleNumber'] = loci[l]['ref_number']
            
        self.loci                = loci
        self.maxSize1read        = maxSize1read
        self.minSize1read        = minSize1read
        self.maxPrimerErrors     = maxPrimerErrors
        self.averagePrimerLength = sum(len(self.loci[l]['ref_forwardP'])+
                                       len(self.loci[l]['ref_reverseP'])
                                       for l in self.loci)/(2*len(self.loci))
        
    def processPrimers(self,withPrimerErrors=0):
        # Find previous readfragments
        self.markpositions('&')
        # Primer errors setup
        if withPrimerErrors:
            # Calculate kmer size
            kmerSize = int(self.averagePrimerLength/(1+withPrimerErrors))  # k is half primer length
            # Kmer count in expected sequences
            self.reference_kmers = {}
            for l in self.loci:
                for seq in (self.loci[l]['ref_sequence'],complement(self.loci[l]['ref_sequence'])):
                    for i in range(len(seq)+1-kmerSize):
                        try: self.reference_kmers[seq[i:i+kmerSize]]+=1
                        except KeyError: self.reference_kmers[seq[i:i+kmerSize]]=1
        # Find primers
        for locus in self.loci:
            for primertype in ('ref_forwardP','ref_reverseP','ref_forwardP_c','ref_reverseP_c'):
                primer = (self.loci[locus][primertype] if not primertype.endswith('_c')
                          else complement(self.loci[locus][primertype[:-2]]))
                if not withPrimerErrors:
                    self.markpositions(primer,locus,primertype)
                else:
                    primerKmers = {i:primer[i:i+kmerSize] for i in range(0,len(primer)+1-kmerSize,kmerSize)}
                    if len(primer)%kmerSize != 0: primerKmers[kmerSize] = primer[-kmerSize:]
                    
                    # (YG) DEBUG
                    print('{} {} {} has {} k-mers (k={})'.format(locus, primertype, primer, len(primerKmers), kmerSize))
                    
                    for o in primerKmers:
                        # (YG) kmer must occur only once in reference sequence (which includes the primer sequences)
                        if self.reference_kmers[primerKmers[o]] == 1:
                            self.markpositions(primerKmers[o],locus,primertype,offset=o)
        # Remove duplicates TODO (make set, then sorted for list)
        # Sort on position
        self.primerPositions.sort(key = lambda x: x[0])

    def markpositions(self,pattern,locus=None,primertype=None,offset=0):
        try:
            currentPosition = self.read.find(pattern)
            while currentPosition != -1:
                self.primerPositions.append((currentPosition-offset,locus,primertype))
                currentPosition = self.read.find(pattern,currentPosition+1)
            
        except AttributeError:
            self.primerPositions = []
            self.markpositions(pattern,locus,primertype)

    def extractReads(self,filterArtefacts=True):
        self.subreads = []         # [((pos, locus, primertype), (pos, locus, primertype)),]
        pp = self.primerPositions  # [(pos, locus, primertype), ]
        for i in range(len(pp)-1):
            if (self.minSize1read < (pp[i+1][0]-pp[i][0]) < self.maxSize1read and   # 70 < ampliconsize < 500
                pp[i][1] == pp[i+1][1] and pp[i][2] and pp[i+1][2] and              # must be same locus, must have primertype
                pp[i][2][:6] != pp[i+1][2][:6] and                                  # cannot be both ref_for or both ref_rev
                (pp[i][2].endswith('_c') ^ pp[i+1][2].endswith('_c'))):             # one must end with _c
                self.subreads.append((pp[i],pp[i+1]))
        if filterArtefacts:
            nonArtifacts   = {('ref_forwardP', 'ref_reverseP_c'),
                              ('ref_reverseP', 'ref_forwardP_c')}
            self.artifacts = [s for s in self.subreads if (s[0][2],s[1][2]) not in nonArtifacts]
            self.subreads  = [s for s in self.subreads if (s[0][2],s[1][2]) in nonArtifacts]
            
            # (YG) DEBUG
            #print('Found {} subreads, {} artifacts'.format(len(self.subreads), len(self.artifacts)))
            #for artifact in self.artifacts:
            #  print('Artifact: {}'.format(artifact))

    def sortsubreads(self):
        # Sort first on length
        self.subreads.sort(key=lambda x: (x[1][0]-x[0][0])+len(self.loci[x[1][1]][x[1][2].replace('_c','')]))
        # Then on locus
        self.subreads.sort(key=lambda x: x[0][1])

    def exportReads(self,filename,mode='wt',type='fasta',locus=None,alleleLength=None,maxReads=None):
        ci = count(1)
        with open(filename,mode) as outfile:
            countBlankInSeq = 0
            for r in self.subreads:
                # (YG) sequence reconstruction: from startpos left primer match to startpos-1 right primer match + sequence of the right primer
                seq = self.read[r[0][0]:r[1][0]+len(self.loci[r[1][1]][r[1][2].replace('_c','')])]
                if ' ' in seq:
                    countBlankInSeq+=1
                    continue
                if 'reverse' in r[0][2]:
                    seq = complement(seq)
                    orientation = 'reverse'
                else: orientation = 'forward'
                outfile.write('>{} {} ({}): {} - {} ({})\n'.format(next(ci),r[0][1],len(seq),r[0][0],r[1][0],orientation))
                outfile.write(seq+'\n')
            if countBlankInSeq: print(countBlankInSeq,"reads contained blanks and were not exported")

    def histLengths(self):
        self.sortsubreads()
        self.histLengthData = {}
        for subr in self.subreads:
            locus = subr[0][1]
            # (YG) sequence reconstruction: from startpos left primer match to startpos-1 right primer match + sequence of the right primer
            seq = self.read[subr[0][0]:subr[1][0]+len(self.loci[locus][subr[1][2].replace('_c','')])]
            #if self.loci[locus]['locusType']:
            #    length = float(calculateAlleleNumber(seq,self.loci[locus]))
            #else:
            length = len(seq)
            try:
                self.histLengthData[locus][length]+=1
            except KeyError:
                if locus not in self.histLengthData: self.histLengthData[locus] = {}
                self.histLengthData[locus][length]=1

    def peakDetection(self,peak_min_width=8,peak_max_width=12,
                      peak_max_hight_diff=0.2,
                      includeRefProfile=False):
        from scipy import signal
        from scipy.optimize import curve_fit
        import numpy as np
        
        self.profile  = {}
        plotDimension = np.sqrt(len(self.histLengthData))
        fig,axes      = plt.subplots(ceil(plotDimension),ceil(len(self.histLengthData)/plotDimension),sharex=True,sharey=True)
        fig.set_size_inches(2*11.692, 2*8.267)
        
        for l,ax in zip(sorted(self.histLengthData),
                        (ax for row in axes for ax in row)):
            locusType = self.loci[l]['locusType']
            x         = sorted(self.histLengthData[l])
            x_range   = list(range(0,x[-1]+1))
            y         = [0 if i not in x else self.histLengthData[l][i] for i in x_range]
            peaks     = signal.find_peaks_cwt(y,np.arange(peak_min_width,peak_max_width))
            peaks     = [(x_range[p],y[p]) for p in peaks]
            peaks.sort(key = lambda x: x[1],reverse = True)
            
            # Potential peaks
            if len(peaks) > 1 and peaks[0][1]*peak_max_hight_diff < peaks[1][1]:
                peaks = peaks[:2]
            else:
                peaks = [peaks[0]]
                
            # Calculate gaussian fit
            x = np.array(x)
            y = np.array([self.histLengthData[l][i] for i in x])
            ax.plot(x,y,'b+',label=l, marker='.')
            try:
                if len(peaks) == 1:
                    popt,pcov       = curve_fit(gaus,x,y,p0=[max(y),peaks[0][0],peak_min_width])
                    self.profile[l] = ((calculateAlleleNumber(' '*int(round(popt[1])),self.loci[l]) if locusType
                                        else round(popt[1]), popt[2]/(locusType if locusType else 1)),)
                
                    ax.plot(x,gaus(x,*popt),'ro:',label='fit1', marker='.')
                elif len(peaks) == 2:
                    popt,pcov = curve_fit(gaus2,x,y,p0=[max(y),peaks[0][0],peak_min_width,
                                                        max(y),peaks[1][0],peak_min_width])
                    self.profile[l] = ((calculateAlleleNumber(' '*int(round(popt[1])),self.loci[l]) if locusType
                                        else round(popt[1]), popt[2]/(locusType if locusType else 1)),
                                       (calculateAlleleNumber(' '*int(round(popt[4])),self.loci[l]) if locusType
                                        else round(popt[4]), popt[5]/(locusType if locusType else 1)))
                    ax.plot(x,gaus2(x,*popt),'ro:',label='fit2', marker='.')
                else: self.profile[l] = None
            except RuntimeError:
                self.profile[l] = None
                
            if includeRefProfile and locusType:
                # Mark the reference profile allele lengths with vertical green lines
                for ra in self.referenceProfile[l]:
                    position = calculateOriginalLength(ra, locusType,
                                                       self.loci[l]['ref_length'],
                                                       self.loci[l]['ref_number'])
                    ax.plot((position,position), ax.get_ylim(),'g-')
                    
            ax.legend()
        plt.show(block=False)

    def CPI(self,populationFile):
        # Reprocess self.profile to ranges of alleles
        self.profileCI = {}
        self.CPI_value = 1
        self.populationData = pd.read_csv(populationFile)
        self.CPI_unusedLoci = set(self.profile) - set(self.populationData['#Locus name'])        
        
        for l in self.profile:
            if l in self.CPI_unusedLoci: continue
            locusAlleles = self.populationData[self.populationData[
                self.populationData.columns[0]]==l]
            locusAN = locusAlleles["Allele number"]
            # TODO float/allele number issue
            alleleRanges = [(float(g[0])-g[1],float(g[0])+g[1]) for g in self.profile[l]]
            if len(alleleRanges) == 2:
                alleleRanges.sort(key = lambda x: x[0])
                if alleleRanges[0][1] > alleleRanges[1][0]:
                    mean = (alleleRanges[0][1] + alleleRanges[1][0])/2
                    alleleRanges = [(alleleRanges[0][0],mean),(mean,alleleRanges[1][1])]
                    # TODO check if with 'mean' is best strategy
            self.profileCI[l] = alleleRanges
            af1 = locusAlleles[(locusAN >= alleleRanges[0][0]) &
                               (locusAN < alleleRanges[0][1])]["Allele Frequency"].sum()
            if len(alleleRanges) == 1:
                locusProbability = af1**2
            else:
                af2 = locusAlleles[(locusAN >= alleleRanges[1][0]) &
                                   (locusAN < alleleRanges[1][1])]["Allele Frequency"].sum()
                locusProbability = 2*af1*af2
            self.CPI_value*=locusProbability

    def linkReferenceProfile(self,referenceProfileFile):
        self.referenceProfile = {}
        with open(referenceProfileFile) as inprofile:
            for line in inprofile:
                if line.startswith('#'): continue
                line = line.strip().split(',')
                self.referenceProfile[line[0]] = (float(line[1]),float(line[2]))
                
# Processing reads
fastq    = open("/media/genomics/raw_data/Nanopore_data/ligatedSTR.fasta")
c        = count(0)
reads    = [line for line in fastq if next(c)%2 == 1]
allreads = '&'.join(reads)

#ligread = LigatedRead('&'.join(reads),loci)
ligread = LigatedRead(allreads,loci)
#ligread.linkReferenceProfile('/media/sf_vm_shared/nanopore/Profile9948A')
ligread.linkReferenceProfile(eu('~/repos/Nanofore/Profile9948A'))
ligread.processPrimers(withPrimerErrors=1)
ligread.extractReads()
ligread.histLengths()
ligread.exportReads('/Nanopore_data/ligatedSTR.fastq/STR.fasta')
ligread.peakDetection(includeRefProfile=True)
ligread.CPI(populationFile = '/home/senne/nanopore/SNP/Europe_SNP_freq.csv')
print("RPM value profile:", ligread.CPI_value)

# (YG) Display profile
print()
print('Estimated profile:')
for locus in sorted(ligread.profile):
    alleles = ':'.join([str(a[0]) for a in ligread.profile[locus]])
    print('  {} {}'.format(locus, alleles))

# Calculate an original length
#calculateOriginalLength(10.3,
                       # ligread.loci['D13S317']['locusType'],
                        #ligread.loci['D13S317']['ref_length'],
                        #ligread.loci['D13S317']['ref_number'])


##PART 2B: Sequence based profiling 

- Mapping using BWA and SAM
- Extracting the uniquely mapped and once with a mapping score 0


In [ ]:
# Init

strFile   = '/media/genomics/raw_data/STR_database.fasta' 
readFile  = '/media/genomics/raw_data/Nanopore_data/ligatedSTR.fasta
samfile = '/media/genomics/raw_data/STR.sam'
bamfile = '/media/genomics/raw_data/STR.bam'.
bambaifile = '/media/genomics/raw_data/STR.bam.bai'
txtfile = '/media/genomics/raw_data/STR.txt'

bwa       = '/opt/tools/bwa-0.7.15'            # v0.7.5
samtools  = '/opt/tools/samtools-1.3.1'        # v1.3.1
bcftools  = '/opt/tools/bcftools-1.3.1'        # v1.3.1


# Check
!ls {strFile}
print('Number of STRs in reference file:')
!grep -c ">" {strFile}

In [ ]:
# Build index of the references
!{bwa} index {strFile}

# Map reads
!{bwa} mem -x ont2d {strFile} {readFile} > {samfile}

print('done')

In [ ]:
# Make sorted bam and index
!{samtools} view -Sbu {samfile} | {samtools} sort -o {bamfile} -
!{samtools} index {bamfile} {bambaifile}
print('Done')

### Extracting the uniquely mapped and once with a mapping score 0

In [ ]:
%%bash 
cd /home/senne/nanopore/Multiplex/results_26-06-2017/Basecalling_2/STR/
echo -n "" > STR.txt
for i in "Amelogenine_X" "Amelogenine_Y" "D3S1358_13" "D3S1358_14" "D3S1358_15" "D3S1358_16" "D3S1358_17" "D3S1358_18" "D3S1358_19" "D21S11_27" "D21S11_28"  "D21S11_29" "D21S11_30" "D21S11_31" "D21S11_31.2" "D21S11_32" "D21S11_32.2" "D21S11_33.2" "TH01_5" "TH01_6" "TH01_7" "TH01_8" "TH01_9" "TH01_9.3" "TH01_10" "TH01_11" "D16S539_9" "D16S539_10" "D16S539_11" "D16S539_12" "D16S539_13" "D16S539_14" "D16S539_15" "D7S820_7" "D7S820_8" "D7S820_9" "D7S820_10" "D7S820_11" "D7S820_12" "D7S820_13" "D7S820_14" "D13S317_8" "D13S317_9" "D13S317_10" "D13S317_11" "D13S317_12" "D13S317_13" "D13S317_14" "D13S818_15" "D13S818_16" "D18S51_12" "D18S51_13" "D18S51_14" "D18S51_15" "D18S51_16" "D18S51_17" "D18S51_18" "D18S51_19" "D18S51_20" "D5S818_9" "D5S818_10" "D5S818_11" "D5S818_12" "D5S818_13" "D5S818_14" "D5S818_15" "D5S818_16" "D5S818_17" "D5S818_18" "D8S1179_8" "D8S1179_9" "D8S1179_10" "D8S1179_12" "D8S1179_13" "D8S1179_14" "D8S1179_15" "D8S1179_16" "D8S1179_17" "D8S1179_18" "FGA_18" "FGA_19" "FGA_20" "FGA_21" "FGA_22" "FGA_23" "FGA_24" "FGA_25" "FGA_26" "FGA_27" "FGA_28" "TPOX_6" "TPOX_7" "TPOX_8" "TPOX_9" "TPOX_10" "TPOX_11" "TPOX_12" "TPOX_13" "vWA_15" "vWA_16" "vWA_17" "vWA_18" "vWA_19" "vWA_20" "SE33_15" "SE33_16" "SE33_18" "SE33_19" "SE33_20" "SE33_21" "SE33_21.2"  "SE33_22.2" "SE33_23.2" "SE33_24.2" "SE33_25.2" "SE33_26.2" "SE33_27.2" "SE33_28.2" "SE33_29.2" "SE33_30.2" "SE33_31.2"; do 
    echo -n "$i;" >> STR.txt
    awk '$2 < 1' STR.sam | awk '$5 > 0'| cut -f3 -d$'\t'| grep -c $i >> STR-NB04.txt
done
